In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from lib.layers import Dense
from lib.activations import Tanh, Sigmoid
from lib.losses import MSE
from lib.network import Sequential

def gradient_check():
    print("----- Gradient Check -----")
    # Setup
    np.random.seed(21)  # For reproducibility
    x_sample = np.random.rand(1, 2)
    y_sample = np.array([[1]])
    layer = Dense(2, 1)
    loss_func = MSE()
    epsilon = 1e-5

    # Get analytic gradient
    output = layer.forward(x_sample)
    loss_func.loss(y_sample, output)
    error_grad = loss_func.loss_prime(y_sample, output)
    layer.backward(error_grad, 0)  # learning_rate=0 to avoid weight updates
    analytic_grad = layer.dW.copy()

    # Get numerical gradient
    numerical_grad = np.zeros_like(layer.weights)
    it = np.nditer(layer.weights, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        ix = it.multi_index
        
        # J(W + e)
        original_w = layer.weights[ix]
        layer.weights[ix] = original_w + epsilon
        output_plus = layer.forward(x_sample)
        J_plus = loss_func.loss(y_sample, output_plus)

        # J(W - e)
        layer.weights[ix] = original_w - epsilon
        output_minus = layer.forward(x_sample)
        J_minus = loss_func.loss(y_sample, output_minus)
        
        # (J(W+e) - J(W-e)) / 2e
        numerical_grad[ix] = (J_plus - J_minus) / (2 * epsilon)
        layer.weights[ix] = original_w  # Restore
        it.iternext()
    
    # Compare
    diff = np.linalg.norm(analytic_grad - numerical_grad) / np.linalg.norm(analytic_grad + numerical_grad)
    print(f"Difference: {diff}")
    assert diff < 1e-4, "Gradient check failed!"
    print("Gradient check passed!")

gradient_check()

In [ ]:
# Reload modules and setup
import sys
for module_name in list(sys.modules.keys()):
    if module_name.startswith('lib'):
        del sys.modules[module_name]

from lib.network import Sequential
from lib.layers import Dense
from lib.activations import Tanh, Sigmoid
from lib.losses import MSE


# XOR Data
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y = np.array([[0], [1], [1], [0]], dtype=np.float32)

# Build model
model = Sequential()
model.add(Dense(2, 16))
model.add(Tanh())
model.add(Dense(16, 1))
model.add(Sigmoid())
model.use_loss(MSE())

print("XOR Problem Setup Complete")
print(f"Input shape: {X.shape}, Output shape: {y.shape}")


In [ ]:
print("Training XOR Network")
print("=" * 50)
model.train(X, y, epochs=10000, learning_rate=1.0)
print("=" * 50)
print("Training Complete")


In [ ]:
# Final Results
print("\n" + "=" * 50)
print("FINAL RESULTS")
print("=" * 50)

predictions = model.predict(X)
pred_values = np.array([p.flatten()[0] for p in predictions])
rounded_preds = np.round(pred_values)

print("\nRaw Predictions:")
for i, (inp, pred, rounded) in enumerate(zip(X, pred_values, rounded_preds)):
    print(f"  Input {inp} → {pred:.6f} → {int(rounded)}")

print(f"\nFinal Prediction Vector: {rounded_preds}")
print(f"Expected XOR Output:    [0. 1. 1. 0.]")
print(f"✓ MATCH: {np.array_equal(rounded_preds, np.array([0., 1., 1., 0.]))}")

# Calculate final loss
final_loss = 0
for i in range(len(X)):
    output = model.predict([X[i]])[0]
    final_loss += model.loss.loss(np.array([[y[i][0]]]), output)
final_loss /= len(X)

print(f"\nFinal Loss: {final_loss:.9f}")
print("=" * 50)
